In [ ]:
import os
from keras import backend as K
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

import densenet
import freezing

os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:
batch_size = 64
nb_classes = 10
nb_epoch = 300

img_rows, img_cols = 32, 32
img_channels = 3

img_dim = (img_channels, img_rows, img_cols) if K.image_data_format() == "channels_first" else (img_rows, img_cols, img_channels)
depth = 40
nb_dense_block = 3
growth_rate = 12
nb_filter = -1
dropout_rate = 0.0  # 0.0 for data augmentation
seed = 333
weight_decay = 0.0001
learning_rate = 0.1

# Preprocessing for DenseNet https://arxiv.org/pdf/1608.06993v3.pdf
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

In [3]:
model = densenet.DenseNet(img_dim, classes=nb_classes, depth=depth, nb_dense_block=nb_dense_block,
                          growth_rate=growth_rate, nb_filter=nb_filter, dropout_rate=dropout_rate, weights=None, weight_decay=1e-4)

model.summary()
sgd = SGD(lr=0.1, momentum=0.9, nesterov=True)  # dampening = 0.9? Should be zero?
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=["accuracy"])
print("Finished compiling")

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#For data preprocessing, we normalize the data using the channel means and standard deviations (https://arxiv.org/pdf/1608.06993v3.pdf)
x_train, x_test = color_preprocessing(x_train, x_test)

x_train45, x_val, y_train45, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=seed)  # random_state = seed


img_gen = ImageDataGenerator(
    horizontal_flip=True,
    width_shift_range=0.125,  # 0.125*32 = 4 so max padding of 4 pixels, as described in paper.
    height_shift_range=0.125,  # first zero-padded with 4 pixels on each side, then randomly cropped to again produce 32×32 images
    fill_mode = "constant",
    cval = 0
)

y_train45 = to_categorical(y_train45, nb_classes)  # 1-hot vector
y_val = to_categorical(y_val, nb_classes)
y_test = to_categorical(y_test, nb_classes)

img_gen.fit(x_train45, seed=seed)

2023-04-26 09:45:16.092437: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 09:45:17.046832: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30961 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0


Model: "densenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 24)   648         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 24)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 24)   0           ['batch_normalization[0][0

/gpfs/space/home/olisavo/.conda/envs/tf_thesis/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [4]:
freezing_list = []
for i in range(len(model.layers)):
  if i < len(model.layers) * 0.8:
    freezing_list.append(int(nb_epoch*0.6))
freezing_list.append(nb_epoch)
checkpointer = ModelCheckpoint('model_dense_c10_best.hdf5', verbose=1, save_best_only=True)

In [5]:
freezing.training_with_freezing(model, img_gen, sgd, x_train45, y_train45, x_val, y_val, x_test, y_test,freezing_list,batch_size=batch_size,lr_schedule = [[0, 0.1],[nb_epoch*0.5,0.01],[nb_epoch*0.75,0.001]],cbks=[checkpointer], name='dense_cifar10')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......activation
.........vars
......activation_1
.........vars
......activation_10
.........vars
......activation_11
.........vars
......activation_12
.........vars
......activation_13
.........vars
......activation_14
.........vars
......activation_15
.........vars
......activation_16
.........vars
......activation_17
.........vars
......activation_18
.........vars
......activation_19
.........vars
......activation_2
.........vars
......activation_20
.........vars
......activation_21
.........vars
......activation_22
.........vars
......activation_23
.........vars
......activation_24
.........vars
......activation_25
.........vars
......activation_26
.........vars
......activation_27
.........vars
......activation_28
.........vars
......activation_29
.........vars
......activation_3
.........vars
......activation_30
.........vars
......activation_31
.........vars
......activation_32
.........vars
......activa

2023-04-25 23:44:10.257176: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200


702/703 [============================>.] - ETA: 0s - loss: 1.5424 - accuracy: 0.4765
Epoch 1: val_loss improved from inf to 1.38849, saving model to model_dense_c10_best.hdf5
703/703 [==============================] - 60s 60ms/step - loss: 1.5419 - accuracy: 0.4768 - val_loss: 1.3885 - val_accuracy: 0.5436 - lr: 0.1000
Epoch 2/180
703/703 [==============================] - ETA: 0s - loss: 1.0485 - accuracy: 0.6593
Epoch 2: val_loss improved from 1.38849 to 0.95965, saving model to model_dense_c10_best.hdf5
703/703 [==============================] - 46s 66ms/step - loss: 1.0485 - accuracy: 0.6593 - val_loss: 0.9597 - val_accuracy: 0.7006 - lr: 0.1000
Epoch 3/180
703/703 [==============================] - ETA: 0s - loss: 0.8552 - accuracy: 0.7284
Epoch 3: val_loss did not improve from 0.95965
703/703 [==============================] - 40s 57ms/step - loss: 0.8552 - accuracy: 0.7284 - val_loss: 1.6539 - val_accuracy: 0.5916 - lr: 0.1000
Epoch 4/180
703/703 [==============================]

[7.569999999999993,
 0.055834416005015326,
 0.32317792477570184,
 0.42384715016430374,
 0.0664976546995827]

In [ ]:
model = densenet.DenseNet(img_dim, classes=nb_classes, depth=depth, nb_dense_block=nb_dense_block,
                          growth_rate=growth_rate, nb_filter=nb_filter, dropout_rate=dropout_rate, weights=None, weight_decay=1e-4)

model.summary()
sgd = SGD(lr=0.1, momentum=0.9, nesterov=True) 
freezing_list = []
for i in range(len(model.layers)):
    if i < len(model.layers) * 0.9:
        freezing_list.append(int(nb_epoch*0.6))
    elif i < len(model.layers) * 0.98:
        freezing_list.append(int(nb_epoch*0.96))
freezing_list.append(nb_epoch)
checkpointer = ModelCheckpoint('model_dense_c10_best_2.hdf5', verbose=1, save_best_only=True)

In [ ]:
freezing.training_with_freezing(model, img_gen, sgd, x_train45, y_train45, x_val, y_val, x_test, y_test,freezing_list,batch_size=batch_size,lr_schedule = [[0, 0.1],[nb_epoch*0.5,0.01],[nb_epoch*0.75,0.001]],cbks=[checkpointer], name='dense_cifar10_2')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......activation
.........vars
......activation_1
.........vars
......activation_10
.........vars
......activation_11
.........vars
......activation_12
.........vars
......activation_13
.........vars
......activation_14
.........vars
......activation_15
.........vars
......activation_16
.........vars
......activation_17
.........vars
......activation_18
.........vars
......activation_19
.........vars
......activation_2
.........vars
......activation_20
.........vars
......activation_21
.........vars
......activation_22
.........vars
......activation_23
.........vars
......activation_24
.........vars
......activation_25
.........vars
......activation_26
.........vars
......activation_27
.........vars
......activation_28
.........vars
......activation_29
.........vars
......activation_3
.........vars
......activation_30
.........vars
......activation_31
.........vars
......activation_32
.........vars
......activa

In [4]:
freezing_list = []
for i in range(len(model.layers)):
    if i < len(model.layers) * 0.9:
        freezing_list.append(int(nb_epoch*0.6))
    elif i < len(model.layers) * 0.98:
        freezing_list.append(int(nb_epoch*0.96))
freezing_list.append(nb_epoch)
checkpointer = ModelCheckpoint('model_dense_c10_best_2_cont.hdf5', verbose=1, save_best_only=True)

In [7]:
freezing.continue_training(model, img_gen, sgd, x_train45, y_train45, x_val, y_val, x_test, y_test,freezing_list,batch_size=batch_size,lr_schedule = [[nb_epoch*0.5,0.01],[nb_epoch*0.75,0.001]],all_epochs=150,weights='dense_cifar10_2_150.h5',cbks=[checkpointer], name='dense_cifar10_2')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......activation
.........vars
......activation_1
.........vars
......activation_10
.........vars
......activation_11
.........vars
......activation_12
.........vars
......activation_13
.........vars
......activation_14
.........vars
......activation_15
.........vars
......activation_16
.........vars
......activation_17
.........vars
......activation_18
.........vars
......activation_19
.........vars
......activation_2
.........vars
......activation_20
.........vars
......activation_21
.........vars
......activation_22
.........vars
......activation_23
.........vars
......activation_24
.........vars
......activation_25
.........vars
......activation_26
.........vars
......activation_27
.........vars
......activation_28
.........vars
......activation_29
.........vars
......activation_3
.........vars
......activation_30
.........vars
......activation_31
.........vars
......activation_32
.........vars
......activa

[7.989999999999995,
 0.058042032223939936,
 0.3539296210247234,
 0.44504994593241937,
 0.06945804049649414]